# Lesson-08 SVM  Bayes Random Forest

implement plagiarism news using SVM

In [1]:
import jieba
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

%matplotlib notebook

In [2]:
corpus = pd.read_csv('/home/student/project/project-01/nlp_bots/hw-xujing/sqlResult_1558435.csv', encoding='gb18030')

In [3]:
corpus.head(1)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm


In [4]:
len(corpus)

89611

In [5]:
news_corpus = corpus.dropna(subset=['source', 'content']) # remove empty news and no source news

In [6]:
content = news_corpus['content'].tolist()
source = news_corpus['source'].tolist()

In [21]:
def label(line):
    if line['source'] == '新华社': la =1
    else: la = 0
    return pd.Series([line['content'], la], index=['content', 'label'])

In [22]:
news_data = news_corpus.apply(label, axis=1)

In [25]:
news_list = news_data.content.tolist()
y = news_data.label.values.astype(np.int)

In [26]:
y.shape

(87052,)

文本预处理

In [27]:
news_cut = []
mask = []
for sen in news_list:
    if not isinstance(sen, str):
        mask.append(Faulse)
        continue
    mask.append(True)
    sentence = ''.join(re.findall(r'[\d|\w]+', sen))
    news_cut.append(" ".join(jieba.cut(sentence)))
    
len(news_cut)

87052

In [28]:
news_cut[0]

'此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息'

In [40]:
vecter =TfidfVectorizer(max_features=300)
X = vecter.fit_transform(news_cut)

In [41]:
X.shape

(87052, 300)

# 模型建立

In [18]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, f1_score, recall_score, roc_auc_score
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [42]:
X_train, x_test, Y_train, y_test = train_test_split(X, y, random_state=20, test_size=0.15)

Naive Bayes

In [43]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, random_state=38, test_size=0.15)

In [44]:
NB = GaussianNB()
NB.fit(x_train.toarray(), y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [46]:
y_pred = NB.predict(x_valid.toarray())
y_pred_pro = NB.predict_proba(x_valid.toarray())

In [48]:
NB.score(x_valid.toarray(), y_valid)

0.8051351351351351

In [49]:
precision_score(y_valid, y_pred)

0.9942225571464456

In [50]:
recall_score(y_valid, y_pred)

0.7889963121698396

In [51]:
f1_score(y_valid, y_pred)

0.8797999444290081

In [53]:
roc_auc_score(y_valid, y_pred_pro[:, 1])

0.9171958404182786

naive bayes 对数据不平衡未作处理， 准确率较平衡截取数据差

# SVM

In [54]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, random_state=199, test_size=0.15)

In [56]:
svc = SVC(verbose=5)
svc.fit(x_train, y_train)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=5)

In [58]:
y_pred= svc.predict(x_valid)


In [59]:
svc.score(x_valid, y_valid)

0.9073873873873873

In [61]:
precision_score(y_valid, y_pred)


0.9073873873873873

In [62]:
recall_score(y_valid, y_pred)

1.0

In [63]:
f1_score(y_valid, y_pred)

0.9514453051199697

In [64]:
roc_auc_score(y_valid, y_pred)

0.5

利用权重， 平衡数据

In [67]:
svc1 = SVC(class_weight='balanced', verbose=5)
svc1.fit(x_train, y_train)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=5)

In [69]:
y_pred = svc1.predict(x_valid)
svc1.score(x_valid, y_valid)

0.8146846846846847

In [70]:
precision_score(y_valid, y_pred)

0.9988796215610606

In [71]:
recall_score(y_valid, y_pred)

0.7966640190627482

In [72]:
f1_score(y_valid, y_pred)

0.8863849765258215

In [73]:
roc_auc_score(y_valid, y_pred)

0.8939545776247593

更换核函数，平衡权重

In [74]:
svc2 = SVC(class_weight='balanced', kernel='linear')
svc2.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [75]:
y_pred = svc1.predict(x_valid)
svc1.score(x_valid, y_valid)

0.8146846846846847

In [76]:
precision_score(y_valid, y_pred)

0.9988796215610606

In [77]:
recall_score(y_valid, y_pred)

0.7966640190627482

In [78]:
f1_score(y_valid, y_pred)

0.8863849765258215

In [79]:
roc_auc_score(y_valid, y_pred)

0.8939545776247593

# random forest

In [80]:
rf = RandomForestClassifier(oob_score=True, class_weight='balanced', random_state=300)
rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=None, oob_score=True,
                       random_state=300, verbose=0, warm_start=False)

In [81]:
y_pred = rf.predict(x_valid)
rf.score(x_valid, y_valid)


0.9915315315315315

In [82]:
precision_score(y_valid, y_pred)

0.9942540122845255

In [83]:
recall_score(y_valid, y_pred)

0.9964257347100873

In [84]:
f1_score(y_valid, y_pred)

0.995338688882277

In [85]:
roc_auc_score(y_valid, y_pred)

0.970002750623526

# 模型效果及优缺点总结

linear regression
优点：
建模速度快，模型解释性容易
缺点：
模型简单对复杂的键值关系拟合较差

logistic regression
优点：模型解释性好，特征权重大，重点特征明显
缺点：不能